#CLEANING AND ASSESSING OSM DATASET

MELAKUKAN DATA ASSESSING PADA DATASET HASIL SCRAPING DARI OSM, BERIKUT HAL YANG SAYA LAKUKAN:


*   Mengambil kolom-kolom *addr:housenumber, addr:street, addr:suburb, addr:city* dan menggabungkannya menjadi 1 kolom yakni kolom *placeAddress* agar sama dengan dataset dari GMAPS (address bersifat opsional agar tidak ada yang *NULL*)
*   Mengambil *latitude* dan *longitude* lokasi dari elemen* node, way, relation* yang berupa *center* atau *geometry* untuk mengisi baris yang kolom *latitude* dan *longitude* nya kosong
* Menambah kolom *placeBussinessStatus* dengan asumsi semua bangunan masih beroperasi atau **OPERATIONAL**
* Menambah kolom *source* untuk label asal data yakni dari **OSM**
* Kolom lain hanya diganti nama kolomnya untuk generalisasi



In [3]:
import pandas as pd
import numpy as np
import ast
import json

ModuleNotFoundError: No module named 'pandas'

In [4]:
with open('/content/drive/MyDrive/BPS_PROJECT/PRE-PROCESSED/osm.json') as f:
    dataOSM = json.load(f)

###PARSING HASIL SCRAPING

In [5]:
places = []
for element in dataOSM['elements']:
    tags = element.get('tags', {})
    place_id = f"osm{element['id']}"
    name = tags.get('name', 'Unknown')

    address_parts = [tags.get(k) for k in ['addr:housenumber', 'addr:street', 'addr:suburb', 'addr:city']]
    address = ', '.join([part for part in address_parts if part])

    lat = (
        element.get('lat') or
        element.get('center', {}).get('lat') or
        (element.get('geometry')[0]['lat'] if element.get('geometry') else None)
    )
    lon = (
        element.get('lon') or
        element.get('center', {}).get('lon') or
        (element.get('geometry')[0]['lon'] if element.get('geometry') else None)
    )

    types = []
    for k in ['amenity', 'shop', 'office']:
        if k in tags:
            types.append(tags[k])
    types.append("point_of_interest")
    types.append("establishment")

    places.append({
        'placeId': place_id,
        'placeName': name,
        'placeBusinessStatus': 'OPERATIONAL',
        'placeAddress': address,
        'placeTypes': ','.join(types),
        'placeLatitude': lat,
        'placeLongitude': lon,
        'source': 'OSM'
    })


In [6]:
dfOSM = pd.DataFrame(places)

In [7]:
dfOSM.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2580 entries, 0 to 2579
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   placeId              2580 non-null   object 
 1   placeName            2580 non-null   object 
 2   placeBusinessStatus  2580 non-null   object 
 3   placeAddress         2580 non-null   object 
 4   placeTypes           2580 non-null   object 
 5   placeLatitude        2580 non-null   float64
 6   placeLongitude       2580 non-null   float64
 7   source               2580 non-null   object 
dtypes: float64(2), object(6)
memory usage: 161.4+ KB


In [8]:
dfOSM.head(20)

,placeId,placeName,placeBusinessStatus,placeAddress,placeTypes,placeLatitude,placeLongitude,source
0,osm456045476,SPBU,OPERATIONAL,"Jl. H. Imam Munandar, Pekanbaru","fuel,point_of_interest,establishment",0.500350,101.471285,OSM
1,osm457837210,Pertamina,OPERATIONAL,,"fuel,point_of_interest,establishment",0.612717,101.428377,OSM
2,osm457837500,Pertamina,OPERATIONAL,,"fuel,point_of_interest,establishment",0.538436,101.402952,OSM
3,osm457837513,Pertamina,OPERATIONAL,,"fuel,point_of_interest,establishment",0.573834,101.398519,OSM
4,osm461577405,RM Sederhana Loket,OPERATIONAL,,"restaurant,point_of_interest,establishment",0.508595,101.444694,OSM
5,osm461802011,Ayam Goreng Suharti,OPERATIONAL,Jalan Riau,"restaurant,point_of_interest,establishment",0.535625,101.432480,OSM
6,osm462161178,Rumah Makan Minan Harapan,OPERATIONAL,,"restaurant,point_of_interest,establishment",0.491418,101.486844,OSM
7,osm462165004,SPBU,OPERATIONAL,,"fuel,point_of_interest,establishment",0.512301,101.395726,OSM
8,osm465135769,Bank Central Asia,OPERATIONAL,,"bank,point_of_interest,establishment",0.521243,101.447123,OSM
9,osm467530312,Pertamina,OPERATIONAL,,"fuel,point_of_interest,establishment",0.464999,101.526956,OSM


In [9]:
dfOSM.columns

Index(['placeId', 'placeName', 'placeBusinessStatus', 'placeAddress',
       'placeTypes', 'placeLatitude', 'placeLongitude', 'source'],
      dtype='object')

In [10]:
dfOSM.to_excel("/content/drive/MyDrive/BPS_PROJECT/PROCESSED/klbi_classification/OSM_FINAL_CLEANED.xlsx")

#CLEANING AND ASSESSING GOOGLE MAPS DATASET

ADDING THE SOURCE COLUMN TO GMAPS DATASET

In [11]:
dataGM = pd.read_excel("/content/drive/MyDrive/BPS_PROJECT/PRE-PROCESSED/SamplePlace_1471.xlsx")

In [12]:
dataGM.head()

,placeId,placeName,placeAddress,placeBusinessStatus,placeTypes,placeLatitude,placeLongitude
0,ChIJG0i9CEap1TERq1G-K3lxuus,Graha Renova,"GC2C+GP3, Tengkerang Barat",OPERATIONAL,"point_of_interest,establishment",0.501254,101.421803
1,ChIJI0kxnE-p1TERVzzCaVLc_-A,Putri Jaya Abadi. CV,"Jalan Musyawarah No.184, Labuh Baru Timur",OPERATIONAL,"furniture_store,home_goods_store,point_of_inte...",0.503969,101.423609
2,ChIJ75xs3BSp1TERjUD6T2Hr7MU,Warung Ojo Forget,"GC3G+GQ4, Jalan Soekarno-Hatta, Sidomulyo Timur",OPERATIONAL,"restaurant,point_of_interest,food,establishment",0.503770,101.426964
3,ChIJn_QB002p1TERjXQjpmobcjE,Batam Central Electronik,"Komplek Taman Anggrek Blok A4-6, Jl. Tuanku Ta...",OPERATIONAL,"electronics_store,point_of_interest,store,esta...",0.503697,101.428838
4,ChIJWxkAUdOp1TERTPhwfB11yBc,BOS (Bintang Oto Service),"Jalan Ikhlas, Labuh Baru Timur",OPERATIONAL,"car_repair,point_of_interest,establishment",0.505233,101.430264


In [13]:
dataGM.columns

Index(['placeId', 'placeName', 'placeAddress', 'placeBusinessStatus',
       'placeTypes', 'placeLatitude', 'placeLongitude'],
      dtype='object')

In [14]:
dataGM.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40763 entries, 0 to 40762
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   placeId              40763 non-null  object 
 1   placeName            40763 non-null  object 
 2   placeAddress         40755 non-null  object 
 3   placeBusinessStatus  40760 non-null  object 
 4   placeTypes           40763 non-null  object 
 5   placeLatitude        40763 non-null  float64
 6   placeLongitude       40763 non-null  float64
dtypes: float64(2), object(5)
memory usage: 2.2+ MB


In [15]:
dataGM['placeAddress'] = dataGM['placeAddress'].fillna('')
dataGM['placeBusinessStatus'] = dataGM['placeBusinessStatus'].fillna('OPERATIONAL')

In [16]:
dataGM.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40763 entries, 0 to 40762
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   placeId              40763 non-null  object 
 1   placeName            40763 non-null  object 
 2   placeAddress         40763 non-null  object 
 3   placeBusinessStatus  40763 non-null  object 
 4   placeTypes           40763 non-null  object 
 5   placeLatitude        40763 non-null  float64
 6   placeLongitude       40763 non-null  float64
dtypes: float64(2), object(5)
memory usage: 2.2+ MB


In [17]:
dataGM['source'] = 'GMAPS'

In [18]:
dataGM.head()

,placeId,placeName,placeAddress,placeBusinessStatus,placeTypes,placeLatitude,placeLongitude,source
0,ChIJG0i9CEap1TERq1G-K3lxuus,Graha Renova,"GC2C+GP3, Tengkerang Barat",OPERATIONAL,"point_of_interest,establishment",0.501254,101.421803,GMAPS
1,ChIJI0kxnE-p1TERVzzCaVLc_-A,Putri Jaya Abadi. CV,"Jalan Musyawarah No.184, Labuh Baru Timur",OPERATIONAL,"furniture_store,home_goods_store,point_of_inte...",0.503969,101.423609,GMAPS
2,ChIJ75xs3BSp1TERjUD6T2Hr7MU,Warung Ojo Forget,"GC3G+GQ4, Jalan Soekarno-Hatta, Sidomulyo Timur",OPERATIONAL,"restaurant,point_of_interest,food,establishment",0.503770,101.426964,GMAPS
3,ChIJn_QB002p1TERjXQjpmobcjE,Batam Central Electronik,"Komplek Taman Anggrek Blok A4-6, Jl. Tuanku Ta...",OPERATIONAL,"electronics_store,point_of_interest,store,esta...",0.503697,101.428838,GMAPS
4,ChIJWxkAUdOp1TERTPhwfB11yBc,BOS (Bintang Oto Service),"Jalan Ikhlas, Labuh Baru Timur",OPERATIONAL,"car_repair,point_of_interest,establishment",0.505233,101.430264,GMAPS


In [19]:
dataGM.to_excel("/content/drive/MyDrive/BPS_PROJECT/PROCESSED/klbi_classification/GM_FINAL_CLEANED.xlsx", index=False)

#MERGING OSM AND GMAPS DATASET

In [20]:
gmaps = pd.read_excel("/content/drive/MyDrive/BPS_PROJECT/PROCESSED/klbi_classification/GM_FINAL_CLEANED.xlsx")
osm = pd.read_excel("/content/drive/MyDrive/BPS_PROJECT/PROCESSED/klbi_classification/OSM_FINAL_CLEANED.xlsx")

In [21]:
combined_df = pd.concat([gmaps, osm], ignore_index=True)

In [22]:
combined_df = combined_df.drop('Unnamed: 0', axis=1)

In [23]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43343 entries, 0 to 43342
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   placeId              43343 non-null  object 
 1   placeName            43343 non-null  object 
 2   placeAddress         41352 non-null  object 
 3   placeBusinessStatus  43343 non-null  object 
 4   placeTypes           43343 non-null  object 
 5   placeLatitude        43343 non-null  float64
 6   placeLongitude       43343 non-null  float64
 7   source               43343 non-null  object 
dtypes: float64(2), object(6)
memory usage: 2.6+ MB


In [24]:
combined_df['placeAddress'] = combined_df['placeAddress'].fillna('-')

In [25]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43343 entries, 0 to 43342
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   placeId              43343 non-null  object 
 1   placeName            43343 non-null  object 
 2   placeAddress         43343 non-null  object 
 3   placeBusinessStatus  43343 non-null  object 
 4   placeTypes           43343 non-null  object 
 5   placeLatitude        43343 non-null  float64
 6   placeLongitude       43343 non-null  float64
 7   source               43343 non-null  object 
dtypes: float64(2), object(6)
memory usage: 2.6+ MB


#DATA ASSESSING

* Memeriksa data dan mengecek apakah missing value, memeriksa apakah ada duplikasi
* Mengisi kolom atau mendrop baris yang memiliki missing value

In [26]:
print("Jumlah duplikasi: ", combined_df.duplicated().sum())

Jumlah duplikasi:  0


In [27]:
combined_df.isna().sum()

,0
placeId,0
placeName,0
placeAddress,0
placeBusinessStatus,0
placeTypes,0
placeLatitude,0
placeLongitude,0
source,0


In [28]:
combined_df.describe()

,placeLatitude,placeLongitude
count,43343.000000,43343.000000
mean,0.498549,101.433454
std,0.039780,0.038976
min,0.420513,101.332527
25%,0.467235,101.406718
50%,0.496659,101.435286
75%,0.524405,101.456568
max,0.680416,101.599236


In [29]:
for col in combined_df.select_dtypes(include='object'):
    combined_df[col] = combined_df[col].str.strip()

In [30]:
mask = combined_df['placeName'].str.contains(r'^[^a-zA-Z]*$', na=False)
df_clean = combined_df[~mask].reset_index(drop=True)

In [31]:
import re
import unicodedata

In [32]:
def normalize_text(text):
    if pd.isna(text):
        return ""

    # Normalisasi unicode ke bentuk NFC
    text = unicodedata.normalize('NFC', text)

    # 1. Hapus karakter khusus termasuk emoji
    text = re.sub(r'[^\w\s.,&+\'-]', ' ', text, flags=re.UNICODE)

    # 2. Hapus angka yang tidak relevan di awal string
    text = re.sub(r'^\d+[\s\-]*', '', text)

    # 3. Standarisasi bentuk usaha
    text = re.sub(
        r'\b(store|toko|shop|warung|resto)\b',
        '',
        text,
        flags=re.IGNORECASE
    )

    # 4. Bersihkan spasi dan format
    text = re.sub(r'\s+', ' ', text).strip()
    return text.lower()

In [33]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43325 entries, 0 to 43324
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   placeId              43325 non-null  object 
 1   placeName            43325 non-null  object 
 2   placeAddress         43325 non-null  object 
 3   placeBusinessStatus  43325 non-null  object 
 4   placeTypes           43325 non-null  object 
 5   placeLatitude        43325 non-null  float64
 6   placeLongitude       43325 non-null  float64
 7   source               43325 non-null  object 
dtypes: float64(2), object(6)
memory usage: 2.6+ MB


In [34]:
combined_df.to_excel("/content/drive/MyDrive/BPS_PROJECT/PROCESSED/klbi_classification/COMBINED_DATASET.xlsx", index=False)

#MENGECEK DUPLIKASI BERDASARKAN LOKASI / JARAK ANTAR POIN

In [35]:
# !pip install fuzzywuzzy rapidfuzz

In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as pyplot
from sklearn.cluster import DBSCAN
from geopy.distance import geodesic
from rapidfuzz import fuzz
from sklearn.neighbors import BallTree

####CLUSTERING BERDASARKAN KEDEKATAN LOKASI DAN NAMA TEMPAT SAMA (MELIHAT JARAK ANTAR DUPLIKASI)

**100M atau 1KM**

In [37]:
def preprocess_data(df):
    # Filter data tidak valid
    valid_mask = (
        df['placeName'].str.contains(r'[a-zA-Z]{3}', na=False) &  # Minimal 3 huruf
        df['placeName'].str.len().between(3, 100) &                # Panjang nama
        df['placeLatitude'].between(-90, 90) &                     # Valid lat
        df['placeLongitude'].between(-180, 180)                    # Valid lon
    )
    df = df[valid_mask].copy()

    # Normalisasi kolom teks
    df['clean_name'] = df['placeName'].apply(normalize_text)

    # Buat spatial bins
    df['lat_bin'] = np.round(df['placeLatitude'], 3)
    df['lon_bin'] = np.round(df['placeLongitude'], 3)

    return df.reset_index(drop=True)

In [38]:
def calculate_similarity(name1, name2):
    # Preprocessing khusus untuk similarity check
    def process(s):
        s = re.sub(r'\b(jl|jalan|no|rt|rw)\b', '', s)  # Hapus alamat umum
        s = re.sub(r'\d+', '', s)                      # Hapus angka
        return s.strip()

    return fuzz.token_set_ratio(process(name1), process(name2))

In [39]:
def spatial_clustering(df, radius=50, name_sim=85):
    clusters = []
    processed = set()

    # Group by spatial bins dan tipe usaha
    for (lat_bin, lon_bin), group in df.groupby(['lat_bin', 'lon_bin']):
        if len(group) < 2:
            continue

        coords = np.deg2rad(group[['placeLatitude', 'placeLongitude']])
        tree = BallTree(coords, metric='haversine')
        neighbors = tree.query_radius(coords, r=radius/6371000)

        for idx, nearby in enumerate(neighbors):
            original_idx = group.index[idx]
            if original_idx in processed:
                continue

            current = group.loc[original_idx]
            cluster = [original_idx]

            for neighbor_idx in nearby:
                neighbor = group.iloc[neighbor_idx]
                if (neighbor.name == original_idx) or (neighbor.name in processed):
                    continue

                # Hitung similarity dengan preprocessing khusus
                sim = calculate_similarity(current['clean_name'], neighbor['clean_name'])

                if sim >= name_sim:
                    cluster.append(neighbor.name)
                    processed.add(neighbor.name)

            if len(cluster) > 1:
                clusters.append(cluster)
                processed.add(original_idx)

    return clusters

In [40]:
def merge_clusters(df, clusters):
    merged = []
    clustered = set()

    for cluster in clusters:
        clustered.update(cluster)
        main = df.loc[cluster[0]].copy()

        # Aggregasi data
        main['placeLatitude'] = df.loc[cluster, 'placeLatitude'].mean()
        main['placeLongitude'] = df.loc[cluster, 'placeLongitude'].mean()
        main['sources'] = ','.join(df.loc[cluster, 'source'].unique())

        merged.append(main)

    # Gabung data tercluster dan non-cluster
    non_cluster = df[~df.index.isin(clustered)]
    return pd.concat([non_cluster, pd.DataFrame(merged)], ignore_index=True)


In [41]:
def final_cleanup(df):
    # Hapus duplikat berdasarkan nama+koordinat
    df = df.drop_duplicates(
        subset=['clean_name', 'lat_bin', 'lon_bin'],
        keep='first'
    )

    # Filter outlier statistik
    coord_cols = ['placeLatitude', 'placeLongitude']
    Q1 = df[coord_cols].quantile(0.25)
    Q3 = df[coord_cols].quantile(0.75)
    IQR = Q3 - Q1

    return df[
        ~((df[coord_cols] < (Q1 - 1.5*IQR)) | (df[coord_cols] > (Q3 + 1.5*IQR)).any(axis=1))
    ]

In [42]:
preprocessed = preprocess_data(combined_df)

# 3. Clustering
clusters = spatial_clustering(
    preprocessed,
    radius=100,    # 50 meter
    name_sim=80   # 85% similarity
)

# 4. Merging
merged = merge_clusters(preprocessed, clusters)


In [ ]:
final_df = final_cleanup(merged)

In [ ]:
final_df.to_excel('./fixed.xlsx', index=False)

In [ ]:
final_df.info()